In [12]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import Sequential
import numpy as np
from keras import Model
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Input
from keras.callbacks import ModelCheckpoint
import pickle
from nltk.tokenize.treebank import TreebankWordTokenizer

In [2]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size, index_from=3)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples
Loaded dataset with 25000 training samples, 25000 test samples


In [3]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [4]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+3) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2

id_to_word = {value:key for key,value in word_to_id.items()}
print(' '.join(id_to_word[id] for id in X_train[0] ))

<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD

In [5]:
training=[]
for i in range(len(X_train)):
    sent =[]
    for j in range(len(X_train[i])):
        sent.append(id_to_word[X_train[i][j]])
    training.append(sent)

In [6]:
embeddings_index = {}
with open('/Users/mohammad/Documents/Internship-IAI/indian hotel/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        
        coefs = [float(i) for i in values[1:]]
        embeddings_index[word] = coefs

embeddings_index['<PAD>'] = [0] * 100
embeddings_index['<UNK>'] = [1] * 100

<h2 style="color:blue">converted to words</h2>

In [7]:
with open("imdb_embedding", "rb") as fp:
    X_train = pickle.load(fp)

In [30]:
X_train = np.array(X_train)

In [ ]:
MAX_SEQ = 500
for s in range(len(X_train)):
    for v in range(len(X_train[s])):
        if X_train[s][v] in embeddings_index:
            X_train[s][v] = embeddings_index[X_train[s][v]]
        else:
            X_train[s][v] = embeddings_index['<UNK>']

In [8]:
BATCH_SIZE = 64
MAX_SEQ = 500
input_layer = Input( batch_shape = (BATCH_SIZE, MAX_SEQ, 100))
lstm_layer = LSTM(100)(input_layer)
output_layer = Dense(1, activation="sigmoid")(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (64, 500, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (64, 100)                 80400     
_________________________________________________________________
dense_1 (Dense)              (64, 1)                   101       
Total params: 80,501
Trainable params: 80,501
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (64, 500, 100)            0         
_________________________________________________________________
lstm_1 (LSTM)                (64, 100)                 80400     
_________________________________________________________________
dense_

In [32]:
checkpoint = ModelCheckpoint('weight_sentiment_dekhte.{epoch:02d}.hdf5')
callbacks_list = [checkpoint]

In [33]:
X_train = X_train[:24896]
y_train = y_train[:24896]

In [ ]:
batch_size = 64
num_epochs = 2
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list)

In [9]:
model.load_weights('weight_sentiment_dekhte.02.hdf5')

In [31]:
BATCH_SIZE = 1
MAX_SEQ = 500
input_layer = Input( batch_shape = (BATCH_SIZE, MAX_SEQ, 100))
lstm_layer = LSTM(100)(input_layer)
output_layer = Dense(1, activation="sigmoid")(lstm_layer)

dekhtemodel = Model(inputs=input_layer, outputs=output_layer)
dekhtemodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
dekhtemodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (1, 500, 100)             0         
_________________________________________________________________
lstm_2 (LSTM)                (1, 100)                  80400     
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    101       
Total params: 80,501
Trainable params: 80,501
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (1, 500, 100)             0         
_________________________________________________________________
lstm_2 (LSTM)                (1, 100)                  80400     
_________________________________________________________________
dense_

In [39]:
we = model.get_weights()
dekhtemodel.set_weights(we)

In [75]:
batch_size = 1
num_epochs = 1
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
dekhtemodel.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list)

Train on 24895 samples, validate on 1 samples
Epoch 1/1
Train on 24895 samples, validate on 1 samples
Epoch 1/1
 4270/24895 [====>.........................] - ETA: 3:24:18 - loss: 0.3577 - acc: 0.8475

KeyboardInterrupt: 

KeyboardInterrupt: 

In [18]:
def senToVe(sent):
    tk = TreebankWordTokenizer()
    sent = tk.tokenize(sent)
    sent = [embeddings_index[sent[-499 + len(sent) - 1 + i]] if i > 499 - len(sent) else embeddings_index['<PAD>'] for i in range(500)]
    sent = np.reshape(sent, (1 , 500 , 100))
    return sent

In [92]:
sent = "it was good"
dekhtemodel.predict(senToVe(sent))

array([[0.6170401]], dtype=float32)

array([[0.6170401]], dtype=float32)